In [85]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error



In [66]:
# Define the ticker symbol for JPMorgan Chase & Co.
ticker_symbol = 'JPM'

# Initialize the yfinance Ticker object for JPM
jpm_ticker = yf.Ticker(ticker_symbol)

# Get available expiration dates
expiration_dates = jpm_ticker.options

# Choose an expiration date that exists in the list of available expiration dates
# For the purpose of this example, let's choose the first available expiration date
expiration_date = expiration_dates[0]

# Download the options data for JPM for the chosen expiration date
jpm_options = jpm_ticker.option_chain(expiration_date)

# Separate the calls and puts data
jpm_calls = jpm_options.calls
jpm_puts = jpm_options.puts

# Display the first few rows of the calls data

df = pd.DataFrame(jpm_calls)

In [68]:
df.head()

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
0,JPM240126C00090000,2024-01-02 16:33:56+00:00,90.0,80.87,79.70,81.35,0.000000,0.000000,1.0,0,3.125002,True,REGULAR,USD
1,JPM240126C00100000,2024-01-24 18:50:36+00:00,100.0,70.45,69.85,71.00,0.089996,0.127908,1.0,5,3.939453,True,REGULAR,USD
2,JPM240126C00105000,2024-01-24 18:49:50+00:00,105.0,65.50,64.75,66.15,0.089996,0.137588,1.0,15,3.804688,True,REGULAR,USD
3,JPM240126C00130000,2024-01-03 15:23:05+00:00,130.0,40.80,39.70,40.90,0.000000,0.000000,NaN,0,2.121098,True,REGULAR,USD
4,JPM240126C00135000,2023-12-26 14:44:24+00:00,135.0,32.96,34.55,36.15,0.000000,0.000000,1.0,0,2.074224,True,REGULAR,USD


### Kraken API Request

In [74]:
import requests
import pandas as pd
from datetime import datetime, timedelta

def fetch_historical_data(pair, interval, since):
    url = f'https://api.kraken.com/0/public/OHLC?pair={pair}&interval={interval}&since={since}'
    response = requests.get(url).json()
    return pd.DataFrame(response['result'][pair], columns=['Time', 'Open', 'High', 'Low', 'Close', 'VWAP', 'Volume', 'Count'])

def get_unix_timestamp(date):
    return int(datetime.timestamp(date))

# Define your pair and interval
pair = 'XXBTZUSD'  # Example for Bitcoin to USD
interval = 1440  # Daily data (1440 minutes in a day)

# Define the date range
end_date = datetime.now()
start_date = end_date - timedelta(days=2*365)  # 2 years back from today
since = get_unix_timestamp(start_date)

# Fetch the historical data
historical_data = fetch_historical_data(pair, interval, since)
historical_data['Close'] = historical_data['Close'].astype(float)

# Convert timestamp to readable date format for clarity
historical_data['Time'] = pd.to_datetime(historical_data['Time'], unit='s')


df = pd.DataFrame(historical_data)

In [76]:
df.head()

,Time,Open,High,Low,Close,VWAP,Volume,Count
0,2022-02-05,41597.2,41926.4,40883.7,41419.0,41509.6,2211.15688361,24916
1,2022-02-06,41419.0,42694.5,41141.9,42419.5,41772.3,1818.07180806,22192
2,2022-02-07,42426.1,44528.6,41697.4,43867.2,43382.3,4604.68183967,39555
3,2022-02-08,43873.1,45500.0,42709.6,44079.1,43881.5,3432.44974988,35663
4,2022-02-09,44083.1,44831.0,43187.2,44420.0,44013.4,3210.42939961,27148


### deribit API Request

In [ ]:
#MaW0sFMebW5vxx_WvodEHmgJ2bcqXLDXmMDgUsieesQ
#Hrykz9ld



In [80]:
import requests
import pandas as pd

# Your API key
api_key = 'MaW0sFMebW5vxx_WvodEHmgJ2bcqXLDXmMDgUsieesQ'

# The base URL for Deribit's API
base_url = 'https://www.deribit.com/api/v2/'

# Headers for authentication
headers = {
    'Authorization': f'Bearer {api_key}'
}

# Define the instrument name
instrument_name = 'BTC-25DEC20-50000-C'

# Function to handle API requests
def get_api_response(url):
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        # Handle HTTP errors
        print(f"Error code {response.status_code}: {response.text}")
        return None

# Fetch details about specific options instruments
instrument_url = f"{base_url}public/get_instrument?instrument_name={instrument_name}"
instrument_data = get_api_response(instrument_url)

# Check if the response contains 'result' key
if instrument_data and 'result' in instrument_data:
    df_instrument = pd.DataFrame([instrument_data['result']])
    print(df_instrument.head())
else:
    print("No 'result' key found in the instrument data response")

# Similar checks should be implemented for trade volumes and last trades...


Error code 400: {"jsonrpc":"2.0","error":{"message":"Invalid params","data":{"param":"instrument_name","reason":"instrument not found"},"code":-32602},"usIn":1706145919309705,"usOut":1706145919309872,"usDiff":167,"testnet":false}
No 'result' key found in the instrument data response


In [65]:
import requests
import pandas as pd
from datetime import datetime

def fetch_historical_data(pair, interval, since):
    url = f'https://api.kraken.com/0/public/OHLC?pair={pair}&interval={interval}&since={since}'
    response = requests.get(url)
    data = response.json()
    
    if data['error']:
        raise Exception(f"Error fetching data: {data['error']}")
    
    return data['result'][pair]

def convert_to_dataframe(data):
    columns = ['Time', 'Open', 'High', 'Low', 'Close', 'VWAP', 'Volume', 'Count']
    df = pd.DataFrame(data, columns=columns)
    df['Time'] = pd.to_datetime(df['Time'], unit='s')
    return df

# Example usage
pair = 'XXBTZUSD'  # Kraken's pair name for Bitcoin to USD
interval = 1440  # Daily data (1440 minutes in a day)
since = int(datetime(2020, 1, 1).timestamp())  # Getting data from Jan 1, 2022

try:
    historical_data = fetch_historical_data(pair, interval, since)
    df = convert_to_dataframe(historical_data)
    print(df.head())  # Display the first few rows of data
except Exception as e:
    print(str(e))


        Time     Open     High      Low    Close     VWAP         Volume  \
0 2022-02-05  41597.2  41926.4  40883.7  41419.0  41509.6  2211.15688361   
1 2022-02-06  41419.0  42694.5  41141.9  42419.5  41772.3  1818.07180806   
2 2022-02-07  42426.1  44528.6  41697.4  43867.2  43382.3  4604.68183967   
3 2022-02-08  43873.1  45500.0  42709.6  44079.1  43881.5  3432.44974988   
4 2022-02-09  44083.1  44831.0  43187.2  44420.0  44013.4  3210.42939961   

   Count  
0  24916  
1  22192  
2  39555  
3  35663  
4  27148  


In [64]:
df

,Time,Open,High,Low,Close,VWAP,Volume,Count
0,2022-02-05,41597.2,41926.4,40883.7,41419.0,41509.6,2211.15688361,24916
1,2022-02-06,41419.0,42694.5,41141.9,42419.5,41772.3,1818.07180806,22192
2,2022-02-07,42426.1,44528.6,41697.4,43867.2,43382.3,4604.68183967,39555
3,2022-02-08,43873.1,45500.0,42709.6,44079.1,43881.5,3432.44974988,35663
4,2022-02-09,44083.1,44831.0,43187.2,44420.0,44013.4,3210.42939961,27148
...,...,...,...,...,...,...,...,...
715,2024-01-21,41649.9,42260.0,41482.8,41554.7,41687.0,892.75638839,10991
716,2024-01-22,41554.8,41654.2,39410.9,39510.0,40323.0,4093.73353976,35136
717,2024-01-23,39510.0,40100.0,38518.0,39844.0,39231.1,3365.92043035,30988
718,2024-01-24,39844.0,40492.3,39451.1,40069.8,39930.5,2453.93910708,22651


In [6]:
import requests
import pandas as pd
from datetime import datetime

def fetch_historical_data(pair, interval, since):
    url = f'https://api.kraken.com/0/public/OHLC?pair={pair}&interval={interval}&since={since}'
    response = requests.get(url)
    data = response.json()
    return data['result'][pair]

# Example usage
pair = 'XXBTZUSD'  # Bitcoin to USD
interval = 1440  # Daily data, intervals are in minutes
since = int(datetime(2022, 1, 1).timestamp())  # Start date in UNIX timestamp

historical_data = fetch_historical_data(pair, interval, since)

# Convert to DataFrame for easier handling
df = pd.DataFrame(historical_data, columns=['Time', 'Open', 'High', 'Low', 'Close', 'VWAP', 'Volume', 'Count'])



In [7]:
df.head()

,Time,Open,High,Low,Close,VWAP,Volume,Count
0,1644019200,41597.2,41926.4,40883.7,41419.0,41509.6,2211.15688361,24916
1,1644105600,41419.0,42694.5,41141.9,42419.5,41772.3,1818.07180806,22192
2,1644192000,42426.1,44528.6,41697.4,43867.2,43382.3,4604.68183967,39555
3,1644278400,43873.1,45500.0,42709.6,44079.1,43881.5,3432.44974988,35663
4,1644364800,44083.1,44831.0,43187.2,44420.0,44013.4,3210.42939961,27148
